In [1]:
#imports
%load_ext autoreload
%autoreload 2
import schillinger.harmony as SH
import schillinger.pitch as SP
import numpy as np
import itertools
import random
from music21 import *
%load_ext music21.ipython21

In [2]:
def showNotes(note_sequece, time_meter, duration_divider):
    TS = str(time_meter[0])+'/'+str(time_meter[1])
    s = stream.Score(id='mainScore')
    p0 = stream.Part(id='part0')
    p0.timeSignature = meter.TimeSignature(TS)
    for i, notes in enumerate(note_sequece):
        #nota = pitch.Pitch(notes)
        #nota.octave = 4
        #nota = note.Note(notes)
        #nota.duration.quarterLength = durations[i]*duration_divider
        notes.duration.quarterLength = notes.duration.quarterLength *duration_divider
        p0.append(notes)
    
    s.insert(0, p0)
    return s

In [3]:
def inversions(notes, X=False, Y=False, XY=False, diatonic=False, diatonic_range=7):
    tone_range = 12
    if diatonic:
        tone_range = diatonic_range
    if X:
        return list(reversed(notes))
    elif Y:
        return [((tone_range-x)%tone_range) for x in notes]
    elif XY:
        return list(reversed([((tone_range-x)%tone_range) for x in notes]))
    else:
        print("specify axis!")
        return notes

In [4]:
sequence = [0,3,1,4,0]
sequence

[0, 3, 1, 4, 0]

In [ ]:
durations = [5,1,1,1,8]

In [ ]:
i1 = inversions(sequence, X=True, diatonic=False)
i2 = inversions(sequence, Y=True, diatonic=False)
i3 = inversions(sequence, XY=True, diatonic=False)
i1, i2, i3

In [5]:
def convert_diatonic_to_scale(seq):
    noteseq = []
    sc = scale.MajorScale('C4')
    for s in seq:
        p = pitch.Pitch()
        p.pitchClass = s
        noteseq.append(p)
        #noteseq.append(sc.pitchFromDegree(s+1))
    return noteseq

In [ ]:
seq = convert_diatonic_to_scale(i1)
dur = inversions(durations, X=True)
dur = durations

s = showNotes(seq,dur,[4,4],.5)
s.show('musicxml')

In [ ]:
#### MUSIC21 

In [89]:
#MUSIC21 inversions OLD

#fullInterval = interval.Interval(diatonic=diaInterval, chromatic=chrInterval)
#bInterval = aInterval.complement
#aInterval.intervalClass
#bInterval = aInterval.reverse()
'''
# functions
def reverse_notes(notes):
    return list(reversed(notes))

def mirror_notes_chromatic(notes, center_note):
    new_notes = []
    aNote = center_note
    for i, bNote in enumerate(notes):
        aInterval = interval.notesToInterval(aNote, bNote)
        
        new_pitch = aInterval.transposePitch(aNote.pitch, reverse=True, maxAccidental=1)
        
        new_note = note.Note(new_pitch)
        new_note.duration = bNote.duration
        new_notes.append(new_note)
    return new_notes

def mirror_notes_diatonic(notes, center_note, sc):
    new_notes = []
    aNote = center_note
    for i, bNote in enumerate(notes):
        
        aInterval = interval.notesToInterval(aNote, bNote)
        
        ###
        
        interval_value = aInterval.generic.value
        interval_direction = aInterval.direction
        
        if interval_direction > 0:
            val = -(interval_value-1)
            s = sc.next(aNote, direction=val)
        elif interval_direction < 0:
            val = abs(interval_value+1)
            s = sc.next(aNote, direction=val)
        else:
            s = aNote.pitch
        #print(bNote, s, aInterval, interval_value)
        ###
        new_note = note.Note(s)
        new_note.duration = bNote.duration
        new_notes.append(new_note)
        
    return new_notes

def reverse_intervals_chromatic(notes, center_note):
    new_notes = []
    aNote = center_note
    for i, bNote in enumerate(notes):
        #bNote = n
        aInterval = interval.notesToInterval(aNote, bNote)
        
        bInterval = aInterval.complement
        new_pitch = bInterval.transposePitch(aNote.pitch, reverse=False, maxAccidental=1)
        
        new_note = note.Note(new_pitch)
        new_note.duration = bNote.duration
        new_notes.append(new_note)
    return new_notes

def reverse_intervals_diatonic(notes, center_note, sc):
    new_notes = []
    aNote = center_note
    for i, bNote in enumerate(notes):
        
        aInterval = interval.notesToInterval(aNote, bNote)
        
        aInterval = aInterval.complement
        interval_value = aInterval.generic.value
        interval_direction = aInterval.direction
        
        if interval_direction > 0:
            val = -(interval_value-1)
            s = sc.next(aNote, direction=val)
        elif interval_direction < 0:
            val = abs(interval_value+1)
            s = sc.next(aNote, direction=val)
        else:
            s = aNote.pitch
        #print(bNote, s, aInterval, interval_value)
        new_note = note.Note(s)
        new_note.duration = bNote.duration
        new_notes.append(new_note)
    return new_notes

mirrored_notes = mirror_notes_diatonic(note_seq, note_seq[0], sc)

reversed_notes = reverse_notes(note_seq)

reversed_and_mirrored = reverse_notes(mirrored_notes)

reversed_intervals_ditonic = reverse_intervals_diatonic(reversed_notes, note_seq[0],sc) # ????

seq = reversed_intervals_ditonic

''';

In [92]:
#MUSIC21 inversions 

def music21_permutaions(notes):
    
    return list(itertools.permutations(notes))


def note_inversions(notes, 
                    center_note, 
                    reverse_notes=False, 
                    mirror=False, 
                    reverse_intervals=False, 
                    diatonic=False, scala=None):
    
    new_notes = []
    aNote = center_note
    
    for i, bNote in enumerate(notes):
        
        aInterval = interval.notesToInterval(aNote, bNote)
        
        if reverse_intervals:
            aInterval = aInterval.complement
            
        if diatonic:
            if scala == None:
                print("NEED A SCALE!")
                return None
            
            interval_value = aInterval.generic.value
            interval_direction = aInterval.direction

            if interval_direction > 0:
                val = -(interval_value-1)
                new_pitch = scala.next(aNote, direction=val)
            elif interval_direction < 0:
                val = abs(interval_value+1)
                new_pitch = scala.next(aNote, direction=val)
            else:
                new_pitch = aNote.pitch
        else:
            new_pitch = aInterval.transposePitch(aNote.pitch, reverse=mirror, maxAccidental=1)
            
        #print(new_pitch)
        new_note = note.Note(new_pitch)
        new_note.duration = bNote.duration
        new_notes.append(new_note)
        
    if reverse_notes:
        new_notes = list(reversed(new_notes))
        
    return new_notes

### basics

def convert_pitches_to_notes(notes, durations):
    note_seq = []
    for i, n in enumerate(notes):
        nota = pitch.Pitch(n)
        nota = note.Note(nota.nameWithOctave)
        nota.duration.quarterLength = note_durations[i]
        note_seq.append(nota)
    return note_seq
######

In [114]:
notes = ["G2","D3","E-3","B3"]
note_durations = [1,1,1,1]

In [ ]:
# for bach!
# ->>> Stream.voicesToParts() !!!!

In [ ]:

notes = ["G4","E-4","F4","G4","C4","F4","E-4","D4","E-4"]
note_durations = [2,2,2,2,2,2,1,1,2,]


In [106]:
sc = scale.DorianScale('C')
#sc.show('musicxml')

In [107]:
note_seq = convert_pitches_to_notes(notes, note_durations)

In [118]:

seq = note_inversions(note_seq, note_seq[0], 
                      mirror=True, 
                      reverse_intervals=False, 
                      diatonic=True, scala=sc, 
                      reverse_notes=True)

s = showNotes(seq, [4,4], 1)

s.show('musicxml')

In [ ]:
'''
motiv A
motiv B gegensatz von motiv A
A+B = A1

'''

In [111]:
# scale tests

sc = scale.ConcreteScale(pitches = ["A-2", "D3", "E-3", "F3","G3","B3"])

In [ ]:
us = environment.UserSettings()
for key in sorted(us.keys()):
    #print(key)
    pass
#environment.set('musicxmlPath', '/Applications/MuseScore 2.app/Contents/MacOS/mscore')